# How is NBA Salary linked to Player Performance?

### The goals of this project:

#### 1. Assess how NBA players' salaries are linked to performance. 
#### 2. Players who are overpaid/underpaid by position
#### 3. Driver better decisions to get quality role players at a low-cost

In [1]:
# Import Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

In [ ]:
#